In [42]:
import pandas as pd
import re
import numpy as np

In [200]:
clubs = pd.read_csv('club_info.csv', index_col=0)
rankings = pd.read_csv('ranking_data.csv', index_col=0)

In [201]:
clubs.head()

,club_name,club_loc,club_id,club_zone
0,209 Water Polo Club,Primary Pool:\r\n\t\t\t\tBuhach Colony High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 31318\n \r...,10008
1,A-Town Water Polo Club,Primary Pool:\r\n\t\t\t\tAtascadero High Schoo...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 30234\n \r...,10008
2,Alumni Water Polo Club,Primary Pool:\r\n\t\t\t\tGranite Bay High Scho...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 23883\n \r...,10008
3,American River Water Polo Club,Primary Pool:\r\n\t\t\t\tRio Americano High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 85\n \r\n\...,10008
4,Arroyo Grande Water Polo,Primary Pool:\r\n\t\t\t\tArroyo Grande High Sc...,\r\n\t\t\t\t \r\n\t\t\t\t\tClub ID: 21063\n \r...,10008


In [202]:
rankings.head()

,Finish,Team,Age,Sex,Year,og_name,Squad,Club,Tie
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,LAGUNA BEACH,0.0
1,1,VANGUARD AQUATIC,12,Boys,2016,VANGUARD AQUATICS,A,VANGUARD,0.0
2,1,LO ANGELE PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,LO ANGELE PREMIER,0.0
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,STANFORD,0.0
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,REGENCY,0.0


## Cleaning club data

In [203]:
clubs['club_id'] = clubs['club_id'].str.replace('Club ID:','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\r','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\n','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\t','').str.strip()
clubs.replace(u'\xa0',u'', regex=True, inplace=True)

In [192]:
clubs['pool_name'] = clubs['club_loc'].str.extract('(?<=\\t)([\w\s]+)(?=\\r)')
clubs['pool_loc'] = clubs['club_loc'].str.extract('(?<=\\t)([0-9]+[\w\s,]+)')

In [193]:
clubs['address'] = clubs['pool_loc'].str.extract('([\w\s]+),')
clubs['address'] = clubs['address'].str.extract('(.+?)(?=\B[A-Z])')

clubs['state'] = clubs['pool_loc'].str.extract('([A-Z]{2})')

clubs['zip'] = np.nan
clubs['zip'] = clubs['pool_loc'].str.extract('([0-9]{5})')

clubs['city'] = clubs['pool_loc'].str.extract('[a-z]([A-Z][a-z]+\s*\w*)')

In [194]:
clubs.replace(u'\t',u'', regex=True, inplace=True)
clubs.replace(u'\r',u'', regex=True, inplace=True)
clubs.replace(u'\n',u'', regex=True, inplace=True)

In [183]:
clubs = clubs.drop(['club_loc', 'pool_loc'], axis=1)

In [195]:
clubs.head()

,club_name,club_loc,club_id,club_zone,pool_name,pool_loc,address,state,zip,city
0,209 Water Polo Club,Primary Pool:Buhach Colony High School1800 Buh...,31318,10008,Buhach Colony High School,"1800 Buhach RoadAtwater, CA",1800 Buhach Road,CA,NaN,Atwater
1,A-Town Water Polo Club,Primary Pool:Atascadero High School1 High Scho...,30234,10008,Atascadero High School,"1 High School HillAtascadero, CA93422",1 High School Hill,CA,93422,Atascadero
2,Alumni Water Polo Club,Primary Pool:Granite Bay High School Pool1 Gri...,23883,10008,Granite Bay High School Pool,"1 Grizzly WayGranite Bay, CA95746",1 Grizzly Way,CA,95746,Granite Bay
3,American River Water Polo Club,Primary Pool:Rio Americano High School4540 Ame...,85,10008,Rio Americano High School,"4540 American River DrSacramento, CA95864",4540 American River Dr,CA,95864,Sacramento
4,Arroyo Grande Water Polo,Primary Pool:Arroyo Grande High School495 Vall...,21063,10008,Arroyo Grande High School,"495 Valley RdArroyo Grande, CA93420",495 Valley Rd,CA,93420,Arroyo Grande


In [196]:
clubs['loc'] = clubs['address'] + ", " + clubs['city'] + ", " + clubs['state'] + " " + clubs['zip']

In [197]:
clubs[clubs['loc'].isnull()]

,club_name,club_loc,club_id,club_zone,pool_name,pool_loc,address,state,zip,city,loc
0,209 Water Polo Club,Primary Pool:Buhach Colony High School1800 Buh...,31318,10008,Buhach Colony High School,"1800 Buhach RoadAtwater, CA",1800 Buhach Road,CA,NaN,Atwater,NaN
6,Bakersfield Water Polo Club,Primary Pool:Garces Memorial High School2800 L...,16629,10008,Garces Memorial High School,2800 Loma Linda Dr,NaN,NaN,NaN,NaN,NaN
7,BarDown Water Polo Academy,Primary Pool:CSU StanislausOne University Circ...,31674,10008,CSU Stanislaus,NaN,NaN,NaN,NaN,NaN,NaN
8,Blazer Water Polo Club,Primary Pool:Golden West High School1717 N. Mc...,16120,10008,Golden West High School,1717 N,NaN,NaN,NaN,NaN,NaN
9,Bulldog Water Polo Club,Primary Pool:Fresno State Aquatics Complex5305...,31612,10008,Fresno State Aquatics Complex,"5305 N Campus Dr, M",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
527,North Texas Thunder Water Polo Club,Primary Pool:Carrol ISD Aquatics Center1501 W....,21327,10004,Carrol ISD Aquatics Center,1501 W,NaN,NaN,NaN,NaN,NaN
528,Oklahoma Trident Aquatics Club,Primary Pool:Claremore Recreation Center595 Ve...,34298,10004,Claremore Recreation Center,595 Veteran,NaN,NaN,NaN,NaN,NaN
530,Rio Grande Valley Water Polo Club,Primary Pool:Pharr Aquatic Center1000 S Fir St...,30649,10004,Pharr Aquatic Center,1000 S Fir St,NaN,NaN,NaN,NaN,NaN
535,Trident Water Polo,"Primary Pool:Rogers Natatorium, St. Marks Scho...",30238,10004,NaN,10600 Preston Rd,NaN,NaN,10600,NaN,NaN


In [199]:
clubs.iloc[6,1]

'Primary Pool:Garces Memorial High School2800 Loma Linda Dr.Bakersfield, CA93312'

replace .